 ### OTP Verification System

 ##### 1. Functions

In [8]:

from random import randint 

#1.function for 6 digit otp generator

def otp_code_generator():

    code_str = ''
    
    for n in range(6):
        code_str += str(randint(0,9))
        
    
    #print(code_str)
    return code_str 


In [9]:
#2.OTP verfier

def otp_verifier(gen_otp,user_input):
    if gen_otp == user_input:
        return True
    else:
        return False

In [10]:

#3.function for sending email

import smtplib #simple mail tranfer protocal library

import getpass #for password inputs

#app_email = getpass.getpass("enter application email ") 
#app_pass = getpass.getpass("enter application password") 

with open('app_email_password.txt','r') as appkeys:
    key_list = list(appkeys.read().split(','))
    
app_email = key_list[0]
app_pass = key_list[1]
    

def send_otp_to_email(otp_code,email):
    global app_email
    global app_pass
    
    smtp_object = smtplib.SMTP('smtp.gmail.com',587) # connecting gmails smtp server 
    
    #smtp_object.ehlo() #Connection check
    
    smtp_object.starttls() #start tls


    from_email = app_email
    password = app_pass    
    smtp_object.login(from_email,password)   #gmail login with app password
    
    from_address = from_email
    to_address = email
    subject = "Python OTP verification system"
    message = f'Your One Time Password for email verification is {otp_code}'
    msg = 'Subject:'+subject+'\n'+ message
    smtp_object.sendmail(from_address,to_address,msg)
    smtp_object.quit() #quits session
    return 'OTP sent !'



##### 2.TKinter GUI and control flow

In [11]:
from tkinter import *
import tkinter as tk
from tkinter.ttk import *

#GUI structure

attempt = 0
otp = ''

gui_window = Tk()
gui_window.title('OTP Verification System') 

#creating labels
header_label = Label(gui_window,text="OTP Verification System")
email_label = Label(gui_window,text="Email : ")
otp_label = Label(gui_window,text="Enter 6 digit OTP : ")
email_response = Label(gui_window,text='please enter your email !')
otp_response = Label(gui_window,text='')

#creating input textboxes

email_text = Entry(gui_window,width = 30)
otp_text = Entry(gui_window,width = 30)

#button onclick functions

def generate_otp_clicked():
    
    if len(email_text.get())==0:
        email_response.config(text=f'email not entered!',foreground='red')
    else:
        email_response.config(text='sending otp...')
        global otp
        otp = otp_code_generator()
        generate_otp.config(state=tk.DISABLED)
        email_text.config(state=tk.DISABLED)
        
        try:
            response = send_otp_to_email(otp,email_text.get())
            email_response.config(text=response,foreground='black')
            verify_otp.config(state=tk.NORMAL)
            otp_text.config(state=tk.NORMAL)
            otp_label.grid(row=3,column=0,pady=2,padx=10)
            otp_text.grid(row=3,column=2,pady=2,padx=10)
            verify_otp.grid(row=4,padx=10,pady=10)
            otp_response.grid(row=4,column=2,pady=2,padx=2)
            otp_text.delete(0,'end')
            
        except:
            generate_otp.config(state=tk.NORMAL)
            email_text.config(state=tk.NORMAL)
            email_text.delete(0,'end')
            email_response.config(text='invalid email !',foreground='red')

def verify_otp_clicked():
    global attempt
    global otp
    is_numeric = True
    user_input = otp_text.get()
    for num in user_input:
        
        if num not in ['0','1','2','3','4','5','6','7','8','9']:
            is_numeric = False 
            
    if not is_numeric:
        otp_response.config(text = 'invalid input please enter digits 0-9',foreground='red')
       
    elif len(user_input)<6:
            otp_response.config(text ="entered code is less than 6 digits",foreground='red')
            
    else:
        
        attempt += 1   
        
        if otp_verifier(otp,otp_text.get()):
            otp_response.config(text="correct otp!!!",foreground='green')
            verify_otp.config(state=tk.DISABLED)
            otp_text.delete(0,'end')
            otp_text.config(state=tk.DISABLED)
            reset_button.grid(row=5,column=2,padx=2,pady=5)                
            reset_button.config(state=tk.NORMAL)

        else:
            if attempt == 3:
                otp_response.config(text='3 incorrect attempts!!',foreground='red')
                verify_otp.config(state=tk.DISABLED)
                otp_text.delete(0,'end')
                otp_text.config(state=tk.DISABLED)
                reset_button.grid(row=5,column=2,padx=2,pady=5)
                reset_button.config(state=tk.NORMAL)
            
            else:
                otp_response.config(text=f'incorrect otp attempt no {attempt}!!',foreground='red')
            
    
def reset_clicked():
    global attempt
    global otp
    
    attempt = 0
    otp = ''
    otp_label.grid_forget()
    otp_text.grid_forget()
    verify_otp.grid_forget()
    otp_response.grid_forget()
    reset_button.grid_forget()
    reset_button.config(state=tk.DISABLED)
    generate_otp.config(state=tk.NORMAL)
    email_text.config(state=tk.NORMAL)
    email_response.config(text='please enter your email !')
    otp_response.config(text='')
    email_text.delete(0,'end')
    
    
    
    
#creating buttons

generate_otp = Button(gui_window,text='Generate OTP',command=generate_otp_clicked)
verify_otp = Button(gui_window,text='Verify',command=verify_otp_clicked)
reset_button = Button(gui_window,text='Resend',command=reset_clicked)


                     

#placing labels,textbox and buttons on screen


email_label.grid(row=1,column=0,pady=10,padx=10)
email_text.grid(row=1,column=2,pady=10,padx=10)
generate_otp.grid(row=2,pady=10,padx=10)
email_response.grid(row=2,column=2,pady=2,padx=2)
 

#creating mainloop
gui_window.mainloop()
